## Step 1: Data Wrangling
Starting with the raw data, unwanted rows and columns are removed

In [212]:
import pandas as pd
import pandas_datareader as pdr
import requests as rq
import fuzzywuzzy
from fuzzywuzzy import fuzz


# load data
df = pd.read_csv('test_data.csv', delimiter=',')

# replace header with first row of data

new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header
# drop unwanted columns
df.drop(df.columns[[1,2,7,8,10,11,12,13,14,15,16,17]], axis=1,inplace=True)
# drop unwanted row
df.drop(df.index[0], inplace=True)

# change type of YEAR column to numeric and to actual value
df['YEAR'] = df['YEAR'].astype(int)
df['YEAR'].replace([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14],[2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2017],inplace=True)




Initially, my thought was that organizations belonging in the public sector will not have a ticker symbol; to minimize error messages when extracting financial data, I filter the dataset and get rid of entities belonging in those organizations.

In [213]:
df['ORGANISATION'].unique()


array(['web', 'financial', 'tech, retail', 'telecoms',
       'government, military', 'government', 'retail', 'academic',
       'energy', 'military', 'healthcare', 'tech',
       'government, healthcare', 'web, gaming', 'gaming', 'media',
       'military, healthcare', 'web, military', 'tech, web', 'transport',
       'web, tech', 'legal', 'app'], dtype=object)

In [235]:
df[~df['ORGANISATION'].isin(["military", "government","healthcare","academic","government, military","government, healthcare"])]
df.head()

,Entity,YEAR,records lost,ORGANISATION,METHOD OF LEAK,DATA SENSITIVITY,ticker
2,AOL,2004,92000000,web,inside job,1,None
3,Automatic Data Processing,2005,125000,financial,poor security,20,ADP
4,Ameritrade Inc.,2005,200000,financial,lost / stolen device,20,None
5,Citigroup,2005,3900000,financial,lost / stolen device,300,C
6,Cardsystems Solutions Inc.,2005,40000000,financial,hacked,300,None


In [215]:
df = df[['Entity','YEAR','records lost','ORGANISATION','METHOD OF LEAK','DATA SENSITIVITY']]

In [216]:
# Querying for ticker. Only get those in NYSE or NASDAQ

def get_symbol(entity):
    url = "http://d.yimg.com/autoc.finance.yahoo.com/autoc?query={}&region=1&lang=en".format(entity)

    result = rq.get(url).json()

    for x in result['ResultSet']['Result']:
        if fuzz.partial_ratio(entity, x['name']) >= 80:
            if x['exchDisp'] == 'NYSE' or x['exchDisp'] == 'NASDAQ':
                return x['symbol'] 

In [217]:
entities = df['Entity'].tolist() #put in list if match >=80

In [218]:
TickerList = []

for y in entities:
    symbol = get_symbol(y)
    TickerList.append(symbol)  

In [234]:
# this is to catch companies not listed/no ticker

stockData = pd.DataFrame(columns = ['Symbol','Date', 'Close'])
for x in TickerList:
    if x is not None:
        try:
            result = pdr.get_data_yahoo(x)
            result = result.reset_index()  
            result['Symbol'] = x
            result = result[['Symbol', 'Date', 'Close']]
            stockData = stockData.append(result)
        except:
            print('ticker not found')
    else:
        pass

ticker not found
ticker not found
ticker not found


In [220]:
# write extracted data to file 

df.insert(loc = 6,column = 'ticker', value = TickerList)
updated_data = pd.merge(df, stockData, left_on='ticker',right_on='Symbol', how='right')
updated_data.to_csv('Lab3_data1.csv',index=False)

## Step 2: Data Manipulation

In [232]:
# load data
df2 = pd.read_csv('Lab3_data1.csv', delimiter=',')


I split the data into two sets of data, records lost and stock price for organizations which incurred those loss.

In [204]:
stockPrice = df2[['Entity','Date','Close']]



In [201]:
recordsLost = df2[['YEAR','Entity','records lost']]



Plotting records lost data first, I see that the most significant record loss were in companies Equifax, JP Morgan, and Heartland; so in general, financial institutions had the most significant record loss. I decided to dig further into these organizations.

Table 0: https://public.tableau.com/views/Lab3_0_0RecordsLost/Sheet1?:embed=y&:display_count=yes&publish=yes


In [231]:
equifax = df2[df2.Entity =="Equifax"]
equifax.to_csv('Lab3Equifax.csv')


After looking further into the above 3 institutions, Equifax had the most significant confirming data; there was a big drop in stockprice in the period of data breach (September, 2017). 

Table 0.1: https://public.tableau.com/views/Lab3_0_1EquifaxStockPriceHistoryClosingPrice/Sheet1?:embed=y&:display_count=yes&publish=yes



 ## Step 3. Visualization Final Version : https://public.tableau.com/views/LabAssignment3_1/Sheet1?:embed=y&:display_count=yes&publish=yes
 
 In this final version, there's a strong correlation between data lost and stock price; confirming that financial market does punish data breach heavily. 
 
 However, in hindsight, this result was biased as it only examined one industry; if I had plotted companies in the public sector (healthcare), I would'have found that the trend is much less severe. From looking at other's charts (plotting data from healthcare industry) in class, I could see that the correlation between stock price and record breach is not strong at all, thereby, refuting the initial claim.

Going in further, I could have examined the stock price trend couple months before and after the breach,just to see if there is a trend in the behavior of the market; for example, how long does it take for the financial market recover from an incident such as data breach, and if the stock price was very optimistic before the incident (on a climbing trend), does the organizatio get punished more heavily in event of a data breach.

For this lab session, I had focused too much on the technical part of the assignment and spend more time on small trivial problems such as formating; more focus should have been put on the overall big picture and the underlying message of the data. I also could have incorporated more features of Tableau for data cleanup and saved time in developing codes in python.

# Resources Used:

Retriving Ticker: https://stackoverflow.com/questions/38967533/retrieve-company-name-with-ticker-symbol-input-yahoo-or-google-api

FuzzyWuzzy to find match: Classmate helped with coding and inserting into table
